In [142]:
from proxy_chrome_browser import get_chromedriver, build_proxy_extension
import moviepy.editor as mpy
import glob
import os
from time import sleep
from tqdm import tqdm

In [143]:
br = get_chromedriver(headless=True,path='/home/miko/Downloads/chromedriver-v-106-0-5249-61')

In [144]:
def find_elm(tag,driver=br,_all=False,_by="css"):
    from selenium.webdriver.common.by import By
    if _by=="css":
        _by=By.CSS_SELECTOR
    elif _by=="link_text":
        _by=By.PARTIAL_LINK_TEXT
    elif _by=="link_text_exact":
        _by=By.LINK_TEXT
        
    try:
        if _all:
            out = driver.find_elements(by=_by,value=tag)
        else:
            out = driver.find_element(by=_by,value=tag)
    except:
        #print('None')
        out = None
    return out

In [145]:
def save_screenshot(elm,filename):
    assert filename[-3:] == "png", "filename should end with png"
    scroll_into_view(elm)
    with open(filename,"wb") as f:
        f.write(elm.screenshot_as_png)

In [146]:
def scroll_into_view(elm):
     #source: -https://stackoverflow.com/questions/25733877/selenium-how-do-you-check-scroll-position
     #loc = elm.location
     #current_view_script = """ var doc = document.documentElement;
     #var left = (window.pageXOffset || doc.scrollLeft) - (doc.clientLeft || 0);
     #var top = (window.pageYOffset || doc.scrollTop)  - (doc.clientTop || 0);
        
     #taking screenshot will scroll into view. IF this stops working can use javascript above
     elm.screenshot_as_png
    
     # scroll a bit to add margin (prevent floating login bar from obscuring elm);
     br.execute_script("window.scrollBy(0,-55)", "")
     sleep(.5)


In [147]:
#set to dark theme
br.get("https://www.reddit.com")

In [149]:

icon = find_elm("button[id='USER_DROPDOWN_ID']")
icon.click()
sleep(.5)
dark_btn = find_elm("button[data-redditstyle='true']")
dark_btn.click()
sleep(.5)
# if for some reason click isnt working. set it manually
#input("Confirm darkmode has been set (press enter)")

In [150]:
post_url = "https://www.reddit.com/r/AskReddit/comments/ypzsqh/men_of_reddit_what_are_some_less_obvious_red/"

In [151]:
br.get(post_url)

In [152]:
#br.switch_to.active_element.text

In [153]:
# expand replies
num_replies_threshold = 90
get_replies = True
num_total_loops = 5
current_loop_num = 0
clicked_links = []
while get_replies:
    more_replies_divs = find_elm("div[id^='moreComments']",_all=True)
    expandable_divs = []
    for div in more_replies_divs:
        more_replies_link = find_elm("p",driver=div)
        #p = br.execute_script("return arguments[0].parentNode;",more_replies_link)
        if not more_replies_link:
            continue
        if more_replies_link.text == '' or 'loading..' in more_replies_link.text:
            continue
        num_replies = int(more_replies_link.text.split()[0].replace(",",""))
        if num_replies >= num_replies_threshold:
            if more_replies_link not in clicked_links:
                expandable_divs.append(more_replies_link)
            else:
                continue
    for div in expandable_divs:
        #link_elm = find_elm("div[data-testid^='search-scope-pill-text']")
        try:
            scroll_into_view(more_replies_link)
            div.click()
            sleep(1)
        except:
            continue  
    if len(expandable_divs) == 0:
        get_replies = False
    if current_loop_num >= num_total_loops:
        get_replies = False
    else:
        current_loop_num += 1
    print("getting more replies")
    #todo: find a better stopping condition

getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


In [129]:
# get comments
comments = find_elm("div[class*='Comment']",_all=True)

In [130]:
#prepend original post to comments
post = [find_elm("div[data-test-id='post-content']"),]
post.extend(comments)
comments = post

In [154]:
len(comments)

500

In [155]:
submission_title = post_url.split("/")[-2]
#asset_dir="assets/demo_png_2_vid"
asset_dir="/mnt/n/data/reddit-yt-tts/queued"
completed_dir="/mnt/n/data/reddit-yt-tts/completed"
#thread_ids = []

In [156]:
def save_comment_text(filename,comment):
    comment_text = " ".join([x.text for x in find_elm("p",driver=comment,_all=True)])
    
    with open(filename,"w") as f:
        f.write(comment_text)

In [159]:
#save threaded comment screenshot as png to directory
vote_count_threshold = 100
thread_count = 0
parent_comment_stack = [dict(comment=comments[0],saved=True)]
prev_comment_lvl = 1
prev_comment = comments[0]
prev_comment_id = -1
i = 0
for comment in tqdm(comments):
    # filter deleted comments
    if "Comment deleted by user" in comment.text:
        continue
    
    # some comments arnt expanded, skipping
    if len(comment.text.split("\n")) <= 4:
           continue
           
    #filter highly voted comments
    vote_div = find_elm("div[id^='vote-arrows']",driver=comment)
    multiply_1k = 'k' in vote_div.text
    if multiply_1k:
        vote_count = float(vote_div.text.replace("k","")) * 1000
    else:
        vote_count = int(vote_div.text)
    
    if i == 0:
        comment_lvl = 1
    else:
        comment_lvl = find_elm("span",comment).text
        comment_lvl = int(comment_lvl.lower().replace("level","").strip())

    #if "1" in comment_lvl:
    if comment_lvl == 1:
        thread_count += 1
        user = comment.text.split("\n")[1]
        vote_count_txt = vote_div.text.replace(".","_")
        thread_id = f"{thread_count:04}-{user}-{vote_count_txt}"
        
        comment_id=0
    else:
        comment_id+=1
        
    # Save parent comments even if threshold is not met
    if comment_lvl > prev_comment_lvl:
        # entering a nested thread, mark new parent...
        parent_comment_stack.append(dict(comment=prev_comment,comment_id=prev_comment_id,saved=False))
    elif comment_lvl < prev_comment_lvl:
        # leaving a nested thread, remove previous parent
        parent_comment_stack = parent_comment_stack[:-1]
        
    
    # Check whether comment should be saved
    save_comment = False
    if vote_count >= vote_count_threshold:
        save_comment = True
    
    #save parent comment
    parent_comment = parent_comment_stack[-1]
    if save_comment and not parent_comment["saved"]:
        #todo refactor to fn
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(parent_comment['comment'],f"{folder}/{parent_comment['comment_id']:04}.png")
        save_comment_text(filename=f"{folder}/{parent_comment['comment_id']:04}.txt",comment=parent_comment['comment'])
        parent_comment["saved"] = True
        
    #Save comment
    if save_comment:
        #thread_ids.append(thread_id)
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(comment,f"{folder}/{comment_id:04}.png")
        save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment)
    
    # init vars for next iteration
    prev_comment_lvl = comment_lvl
    prev_comment = comment
    prev_comment_id = comment_id
    i+=1

  0%|                                                   | 0/500 [00:00<?, ?it/s]


MaxRetryError: HTTPConnectionPool(host='localhost', port=41237): Max retries exceeded with url: /session/2dd2e6e9172028b680f1d2fea32394a3/element/67156666-b4fd-4765-9454-dd7d67aa87e8/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8df56080d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [92]:
thread_ids = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/{submission_title}/*")]

In [93]:
def build_thread_vid(thread_id):
    images = glob.glob(f"{asset_dir}/{submission_title}/{thread_id}/*.png")
    audio_file_dir = f"{completed_dir}/{submission_title}/{thread_id}"
    # https://zulko.github.io/moviepy/examples/examples.html
    clips = []
    audio_clips = []
    total_duration = 0
    transition_duration = 0
    for i,image in enumerate(images):
      
        image_file = image.split("/")[-1]
        audio_file = image_file.replace(".png",".wav")
        try:
            audio_clip = mpy.AudioFileClip(f"{audio_file_dir}/{audio_file}").set_start(total_duration)
        except OSError as e:
            print(f"audio file {audio_file_dir}/{audio_file} not found")
            continue
        audio_clips.append(audio_clip)
        audio_duration = audio_clip.duration
        
        clip = mpy.ImageClip(image).set_duration(audio_duration).set_start(total_duration).set_position(("center", "center"))
        bg_clip = mpy.ImageClip("assets/bg-img.png").set_duration(audio_duration).set_start(total_duration)
        
        #clip = clip.set_start(5) # TODO: change duration based on audio
        #clip = clip.resize((1080,1920)) #TODO add tts audio
        clips.append(bg_clip)
        clips.append(clip)
        
        total_duration += audio_duration + transition_duration
  
    if len(clips) > 0 :
        final_audio_clip = mpy.CompositeAudioClip(audio_clips)
        final_clip = mpy.CompositeVideoClip(clips)
        final_clip.audio = final_audio_clip
        final_clip.write_videofile(f"{asset_dir}/{submission_title}/{thread_id}/thread_clip.mp4",fps=30,logger=None)
    

In [94]:
f"{asset_dir}/{submission_title}/{thread_ids[0]}"

'/mnt/n/data/reddit-yt-tts/queued/men_of_reddit_what_are_some_less_obvious_red/0215-Ohgood9002-210'

In [95]:
# Execute TTS at this point (on windows GPU) then run below
input("Execute TTS at this point (on windows GPU) then run below...(press enter)")

Execute TTS at this point (on windows GPU) then run below...(press enter) 


''

In [108]:
# save threads to clips
# vote_div.text
thread_ids = sorted(list(set(thread_ids)))
for thread in tqdm(thread_ids):
    print(thread)
    build_thread_vid(thread)

  0%|                                                    | 0/97 [00:00<?, ?it/s]

0001-Jack3715-3_9k


  1%|▍                                           | 1/97 [00:20<33:16, 20.80s/it]

0002-theloosestofcannons-4_0k


  2%|▉                                           | 2/97 [00:47<38:25, 24.27s/it]

...

audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0155-JamesDeansComb-182/0001.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0155-JamesDeansComb-182/0000.wav not found
0156-extropia-331
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0156-extropia-331/0000.wav not found
0170-Maximum_Passion1865-106
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0170-Maximum_Passion1865-106/0000.wav not found
0171-usmcmech-118


 98%|██████████████████████████████████████████ | 95/97 [10:23<00:00, 16.60it/s]

audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0171-usmcmech-118/0000.wav not found
0182-kalel51-105
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0182-kalel51-105/0000.wav not found
0191-Rdikin-108
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0191-Rdikin-108/0000.wav not found
0192-Brilliant-Champion81-119
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0192-Brilliant-Champion81-119/0000.wav not found
0215-Ohgood9002-210


100%|███████████████████████████████████████████| 97/97 [10:24<00:00,  6.43s/it]

audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0215-Ohgood9002-210/0001.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0215-Ohgood9002-210/0002.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0215-Ohgood9002-210/0007.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/men_of_reddit_what_are_some_less_obvious_red/0215-Ohgood9002-210/0000.wav not found


In [109]:
# save c to 

In [110]:
videos = glob.glob(f"{asset_dir}/{submission_title}/*/*.mp4")

In [111]:
all_video_clips = [mpy.VideoFileClip(video) for video in videos]

In [112]:
videos_w_sound_corrected = []
for clip in all_video_clips:
    clip.audio = clip.audio.fx(mpy.afx.audio_normalize)
    videos_w_sound_corrected.append(clip)

In [113]:
transition = mpy.VideoFileClip("assets/tvstatic-180-720.mp4")

In [114]:
transition = transition.set_duration(.7)
#transition = transition.resize(1080,720) resizing crashes kernal reesizing 
#using ffmpeg on cli instead:  https://trac.ffmpeg.org/wiki/Scaling


In [115]:
final_video = mpy.concatenate_videoclips(videos_w_sound_corrected,method="compose",transition=transition)

In [116]:
#import moviepy.video.fx as vfx
#import moviepy.audio.fx as afx
bg_music = mpy.AudioFileClip("assets/bg-test.mp3")

bg_music = bg_music.fx( mpy.afx.audio_normalize).fx( mpy.afx.volumex, 0.3).fx( mpy.afx.audio_fadein, 1.0)
tts_audio = final_video.audio

In [117]:
bgm_w_tts_audio = mpy.CompositeAudioClip([bg_music,tts_audio])

In [118]:
final_video.audio = bgm_w_tts_audio

In [119]:
final_video.write_videofile(f"assets/{submission_title}.mp4",fps=30,logger=None)

In [120]:
1+1

2

In [87]:
#create transition clips
!ls assets/

 bg-img.jpg
 bg-img.png
 bg-test.mp3
 demo_png_2_vid
 demos
 demo_trts
'Free Glitch Transition Overlay Pack №2 HD 4K-7HP3IWdCc3s.mkv'
'Free VHS overlay effect-KQwJ5AM8zns.f137.mp4.part'
 pngtree-soft-small-bokeh-transparency-background-png-image_2326962.jpg
 reddit_whats_your_most_im_with_the_boomers_on.mp4
 test_final_vid-fast.mp4
 test_final_vid.mp4
'TV static glitch effect                                                                (for editing)-cYgODiuD5co.mp4'
'WYS – Snowman-0WmRB9Iz-30.opus'
